In [10]:
import glob
import h5py
import numpy as np
import array
from os import listdir
from os.path import isfile, join
from datetime import datetime

# set directory containing TANSO-FTS/GOSAT L2 CH4 mixing ratio amount (SWIR) product files
# e.g. "/Users/markegge/ch4/capture/1606280000-01/1606280000-01/GOSATTFTS20150915_02C02SV0260R16062800000.h5"

root_dir = "c:/Users/markegge/work/ch4/capture/1606280000-01/1606280000-01"

def read_file(file_path):
    f = h5py.File(file_path, 'r')
    numObs = f.get('scanAttribute')['numScan'][0]
    lats = f.get('Data').get('geolocation')['latitude'][:]
    lons = f.get('Data').get('geolocation')['longitude'][:]
    heights = f.get('Data').get('geolocation')['height'][:]
    xch4s = f.get('Data').get('mixingRatio')['XCH4'][:]
    times = f.get('scanAttribute')['time'][:]
    epoch_times = []
    for f in times:
        seconds = (datetime.strptime(f, '%Y-%m-%d %H:%M:%S.%f') - datetime(1970, 1, 1)).total_seconds()
        epoch_times.append(seconds)

    vals = np.column_stack((lons, lats, epoch_times, xch4s))
    return vals

h5files = [f for f in listdir(root_dir) if isfile(join(root_dir, f)) and f[-2:] == 'h5']

data = np.empty([1,4])
first = True
for file in h5files:
    if first:
        data = read_file(join(root_dir, file))
        first = False
    else:
        data = np.vstack((data, read_file(join(root_dir, file))))
    print data.shape

# lons (x), lats (y), epoch_times (unix epoch), xch4s (ppmv)
np.savetxt("values.csv", data, delimiter=",", fmt=['%10d', '%3.7f', '%3.7f', '%1.7f']) 
l = data.tolist()
print l[0:5]
flat_list = [item for sublist in l for item in sublist]
print flat_list[:10]
array.array('f', flat_list).tofile(open('gosat.bin', 'wb'))

(194L, 4L)
(502L, 4L)
(849L, 4L)
(1215L, 4L)
(1602L, 4L)
(1943L, 4L)
(2331L, 4L)
(2733L, 4L)
(3050L, 4L)
(3343L, 4L)
(3648L, 4L)
(3865L, 4L)
(4130L, 4L)
(4410L, 4L)
(4744L, 4L)
[[162.880126953125, 59.387386322021484, 1442286695.074, 1.8243350982666016], [159.53147888183594, 54.598876953125, 1442286768.825, 1.8265677690505981], [159.52455139160156, 54.59268569946289, 1442286773.357, 1.820183277130127], [159.5192108154297, 54.588287353515625, 1442286778.044, 1.8425511121749878], [158.09283447265625, 52.14934539794922, 1442286837.89, 1.8393828868865967]]
[162.880126953125, 59.387386322021484, 1442286695.074, 1.8243350982666016, 159.53147888183594, 54.598876953125, 1442286768.825, 1.8265677690505981, 159.52455139160156, 54.59268569946289]
